 - try bigram
 - [x] use pretraining embeddings
 - _! BUG ! different batch sizes will end up with different acc score in testing, inconsistent acc for different batch size_

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
# with open(os.path.join(input_path, 'dev_text.txt'), 'r', encoding='utf-8') as f:
#     dev_text = f.read().strip().split('\n')

# with open(os.path.join(input_path, 'heldout_text.txt'), 'r', encoding='utf-8') as f:
#     heldout_text = f.read().strip().split('\n')

In [3]:
# dev_label_path = os.path.join(input_path,'dev_label.txt')
# with open(dev_label_path, 'r', encoding='utf-8') as f:
#     dev_y = f.read().split('\n')

# Prepare data

In [4]:
# dev_data = pd.DataFrame({'text':dev_text, 'label':dev_y})

# dev_data.to_csv(os.path.join(input_path, 'dev_data.tsv'), sep='\t',index=False)

In [5]:
# test_data = pd.DataFrame({'text':heldout_text})
# test_data.to_csv(os.path.join(input_path, 'test_data.tsv'), sep='\t',index=False)

In [705]:
import paths
import baseline_lstm_model
import oh_lstm_model
import fasttext_model
import residual_lstm_model
import config
import util
import importlib

In [706]:
reload_packages = [paths, baseline_lstm_model, fasttext_model, oh_lstm_model,residual_lstm_model, util, config]
for package in reload_packages:
    importlib.reload(package)
# importlib.reload(data)

In [559]:
config.batch_size

1

# Load data

In [730]:
import torch
from torchtext import data
import random
from torchtext.data import TabularDataset
import torch.nn as nn
import torch.optim as optim
import spacy
import re



SEED = 1234

torch.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
def tokenizer(string): 
    return [word.text.lower() for word in nlp(clean(string))]

def clean(text):
    '''remove non alphanumeric character, remove links'''
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    text = re.sub(r'https?:/\/\S+', ' ', text)
    return text.strip()

TEXT = data.Field(sequential=True, tokenize = tokenizer, include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [815]:
dev_datafields = [("text", TEXT), ("label", LABEL)]
dev_dataset = TabularDataset(
               path=os.path.join(paths.input_path, 'dev_data.tsv'),
               format='tsv',
               skip_header=True,
               fields=dev_datafields)

test_datafields = [("text", TEXT)]
test_dataset = TabularDataset(
           path=os.path.join(paths.input_path, 'test_data.tsv'),
           format='csv',
           skip_header=True,
           fields=test_datafields)

In [816]:
train_dataset, valid_dataset = dev_dataset.split(split_ratio=0.85, random_state = random.seed(SEED), stratified=True)

In [817]:
TEXT.build_vocab(train_dataset, 
                 max_size = 40000, 
                 vectors = "glove.840B.300d",
                 unk_init = torch.Tensor.normal_ # initialize unk and pad with normal distribution
                )
LABEL.build_vocab(train_dataset)

In [818]:
TEXT.vocab.itos[1]

'<pad>'

In [819]:
TEXT.vocab.vectors[TEXT.vocab.stoi['<unk>']]

tensor([-0.1245, -2.0167, -1.8408, -1.8693,  0.5336, -1.1077, -2.1604,  0.2489,
        -0.2139, -1.4939,  0.5098,  0.1470,  1.7412,  0.4003,  0.0142,  0.2372,
         0.1753,  0.5593,  1.4649,  0.1637, -1.5092, -1.2597, -0.4065, -2.4065,
        -1.0229, -0.5841, -0.1961,  0.7232,  0.7993, -0.5898,  0.8280,  1.9995,
        -1.6778, -0.6556, -2.1492, -0.5696, -0.9129,  1.7062, -0.9049, -0.2820,
         0.9540, -1.3509,  0.1597,  0.7766,  0.9011, -0.6364, -0.0637, -0.2093,
        -0.7578,  2.4950, -0.1423, -1.1769,  0.4894, -1.6177,  0.9247, -0.9170,
        -0.0758,  0.0505, -0.4633, -0.2515,  0.8401, -1.2632,  0.7737, -1.4244,
         1.1296,  0.4472, -1.2549, -2.6256, -0.0140, -0.4426, -0.4629,  0.4480,
        -0.5305,  1.5050, -0.0608, -2.0750,  1.3199, -0.8722,  0.9296,  0.7483,
        -0.0807, -0.1050, -2.5114, -0.9549,  1.6605,  0.2417, -1.5093, -1.1866,
         1.1405,  0.5623, -1.0970, -0.3976,  1.0002, -0.3124, -1.0867,  0.3114,
         1.5388, -0.6063,  0.1500,  0.15

In [820]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_dataset, valid_dataset), 
    batch_size = 1,
    device = device,
    sort = False, # whether sort the whole dataset with sortkey
    shuffle = True,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True, #sort by length for padding
    repeat = False)

test_iterator = data.Iterator(
    test_dataset,
    batch_size = 1,
    device = device, 
    sort = False, 
    shuffle = False,
    train = False,
#     sort_key = lambda x: len(x.text),
    sort_within_batch = False, # don't wanna sort in testing set
    repeat = False)

In [821]:
train_dataset[0].text[:5]

['a', 'very', 'well', 'made', 'film']

In [822]:
TEXT.vocab.freqs.most_common(3)

[('the', 22332), ('and', 10907), ('a', 10771)]

In [823]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f51a3985ae8>, {'neg': 0, 'pos': 1})


In [824]:
# m = nn.AdaptiveAvgPool2d((1,None))
# input = torch.randn(123, 2, 256).permute(1,0,2)
# output = m(input).squeeze(1)

In [825]:
# output[0][1]

# Define model

In [826]:
# model = baseline_lstm_model.BaselineLstm(vocab_size=len(TEXT.vocab), 
#                                             embed_size=300, 
#                                             hidden_size=32, 
#                                             output_dim=1,
#                                             nlayers=1,
#                                             bidirectional=True,
#                                             lstm_dropout=0,
#                                             dropout=0.6,
#                                             pad_idx=TEXT.vocab.stoi[TEXT.pad_token],
#                                             train_embedding=False)

In [ ]:
# model = oh_lstm_model.OhLstm(vocab_size=len(TEXT.vocab), 
#                                             embed_size=300, 
#                                             hidden_size=32, 
#                                             output_dim=1,
#                                             nlayers=1,
#                                             bidirectional=True,
#                                             lstm_dropout=0,
#                                             dropout=0.4,
#                                             pad_idx=TEXT.vocab.stoi[TEXT.pad_token],
#                                             train_embedding=False)

In [828]:
# model = fasttext_model.Fasttext(vocab_size=len(TEXT.vocab), 
#                                             embed_size=300,
#                                             output_dim=1,
#                                             dropout1=0.2,
#                                             dropout2=0.1,
#                                             pad_idx=TEXT.vocab.stoi[TEXT.pad_token],
#                                             train_embedding=True)

In [857]:
model = residual_lstm_model.ResidualLstm(vocab_size=len(TEXT.vocab), 
                                                embed_size=300, 
                                                hidden_size=32, 
                                                output_dim=1,
                                                nlayers=1,
                                                bidirectional=True,
                                                lstm_dropout=0,
                                                dropout1=0.0,
                                                dropout2=0.3,
                                                dropout3=0.5,
                                                pad_idx=TEXT.vocab.stoi[TEXT.pad_token],
                                                train_embedding=False)
# 02/04 -> 8975

In [858]:
model

ResidualLstm(
  (embedding): Embedding(23168, 300, padding_idx=1)
  (lstm): LSTM(300, 32, bidirectional=True)
  (fc): Linear(in_features=364, out_features=1, bias=True)
  (globalpooling): AdaptiveAvgPool2d(output_size=(1, None))
  (dropout1): Dropout(p=0.0)
  (dropout2): Dropout(p=0.3)
  (dropout3): Dropout(p=0.5)
)

In [859]:
pretrained_embeddings = TEXT.vocab.vectors

# load embedding
model.embedding.weight.data.copy_(pretrained_embeddings)
# # init unk token
# UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
# model.embedding.weight.data[UNK_IDX] = torch.zeros(200)
# model.embedding.weight.data[TEXT.vocab.stoi[TEXT.pad_token]] = torch.zeros(200)

tensor([[-0.1245, -2.0167, -1.8408,  ..., -1.9087,  2.6203, -0.5614],
        [ 0.1405,  2.6338,  1.4239,  ..., -2.4909,  0.6555, -0.9523],
        [ 0.2720, -0.0620, -0.1884,  ...,  0.1302, -0.1832,  0.1323],
        ...,
        [ 0.2548, -0.1572,  0.6147,  ...,  0.2006, -0.3907, -0.3322],
        [ 0.1896,  0.1672,  0.3407,  ..., -0.3786, -0.0278, -0.1949],
        [ 0.3195,  0.2435, -0.1777,  ..., -0.0243, -0.6011,  0.2368]])

In [860]:
optimizer = optim.Adam(model.parameters(), lr=1e-03)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=5e-4)
criterion = nn.BCEWithLogitsLoss()

best_epoch, best_vali_loss, starting_epoch = 0, 400, 0

In [861]:
residual_lstm_model.run(model, 
                        optimizer, 
                        criterion, 
                        train_iterator, 
                        valid_iterator, 
                        best_epoch=best_epoch,
                        best_vali_loss=best_vali_loss, 
                        DEVICE=device, 
                        start_epoch=starting_epoch,
                       model_prefix='residual_lstm_') #residual_lstm_

### Epoch     0 
Epoch: 0	Batch: 200	Avg-Loss: 0.6958	Avg-Acc: 0.5500 
Epoch: 0	Batch: 400	Avg-Loss: 0.6750	Avg-Acc: 0.5400 
Epoch: 0	Batch: 600	Avg-Loss: 0.6516	Avg-Acc: 0.6250 
Epoch: 0	Batch: 800	Avg-Loss: 0.5503	Avg-Acc: 0.7350 
Epoch: 0	Batch: 1000	Avg-Loss: 0.5353	Avg-Acc: 0.7400 
Epoch: 0	Batch: 1200	Avg-Loss: 0.4539	Avg-Acc: 0.8250 
Epoch: 0	Batch: 1400	Avg-Loss: 0.4628	Avg-Acc: 0.8400 
Epoch: 0	Batch: 1600	Avg-Loss: 0.4740	Avg-Acc: 0.7950 
Train Loss: 0.4165	Train Acc: 0.8135	Val Loss: 0.4327	Val Acc: 0.8100 
Epoch time used:  109.40562725067139 s 
### Epoch     1 
Epoch: 1	Batch: 200	Avg-Loss: 0.3746	Avg-Acc: 0.8700 
Epoch: 1	Batch: 400	Avg-Loss: 0.3294	Avg-Acc: 0.8450 
Epoch: 1	Batch: 600	Avg-Loss: 0.3617	Avg-Acc: 0.8400 
Epoch: 1	Batch: 800	Avg-Loss: 0.4400	Avg-Acc: 0.8250 
Epoch: 1	Batch: 1000	Avg-Loss: 0.3985	Avg-Acc: 0.8200 
Epoch: 1	Batch: 1200	Avg-Loss: 0.3571	Avg-Acc: 0.8150 
Epoch: 1	Batch: 1400	Avg-Loss: 0.3920	Avg-Acc: 0.8500 
Epoch: 1	Batch: 1600	Avg-Loss: 0.4087	

KeyboardInterrupt: 

# Prediction

In [851]:
for epoch in [1, 4]:
    # checkpoint = torch.load("checkpoint.pt")
    model_prediction = residual_lstm_model.ResidualLstm(vocab_size=len(TEXT.vocab), 
                                                embed_size=300, 
                                                hidden_size=32, 
                                                output_dim=1,
                                                nlayers=1,
                                                bidirectional=True,
                                                lstm_dropout=0,
                                                dropout1=0.0,
                                                dropout2=0.3,
                                                dropout3=0.3,
                                                pad_idx=TEXT.vocab.stoi[TEXT.pad_token],
                                                train_embedding=False)
#     model_prediction = oh_lstm_model.OhLstm(vocab_size=22557, 
#                                             embed_size=300, 
#                                             hidden_size=64, 
#                                             output_dim=1,
#                                             nlayers=1,
#                                             bidirectional=True,
#                                             lstm_dropout=0,
#                                             dropout=0.4,
#                                             pad_idx=TEXT.vocab.stoi[TEXT.pad_token],
#                                             train_embedding=False)
    # proceeding from old models
    model_path = os.path.join(paths.output_path, 'residual_lstm_'+str(epoch)+'.pth.tar')
    print("=> loading checkpoint '{}'".format(model_path))
    checkpoint = torch.load(model_path)
    starting_epoch = checkpoint['epoch']
    # best_vali_acc = checkpoint['best_vali_acc']
    val_loss: checkpoint['val_loss']
    val_acc: checkpoint['val_acc']
    model_state_dict = checkpoint['model_state_dict']
    model_prediction.load_state_dict(model_state_dict)
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    best_vali_loss = checkpoint['best_vali_loss']
    best_epoch = checkpoint['best_epoch']
    print("=> loaded checkpoint '{}' (epoch {})"
          .format(model_path, checkpoint['epoch']))
    # del checkpoint, model_state_dict

    best_epoch

    model_prediction.cuda()

    prediction = residual_lstm_model.predict(model_prediction, test_iterator, device)
    prediction_itos = [LABEL.vocab.itos[int(idx)] for idx in prediction]

    with open(os.path.join(paths.output_path, 'heldout_pred_nn_'+'residual_lstm_'+str(epoch)+'.txt'), 'w', encoding='utf-8') as f:
        [f.write(prediction_string+'\n') for prediction_string in prediction_itos]

=> loading checkpoint '../outputs/residual_lstm_1.pth.tar'
=> loaded checkpoint '../outputs/residual_lstm_1.pth.tar' (epoch 1)
0
400
800
1200
1600
=> loading checkpoint '../outputs/residual_lstm_4.pth.tar'
=> loaded checkpoint '../outputs/residual_lstm_4.pth.tar' (epoch 4)
0
400
800
1200
1600


In [814]:
len(test_iterator)

2000

In [399]:
# prediction_itos = [LABEL.vocab.itos[int(idx)] for idx in prediction]

In [676]:
# with open(os.path.join(paths.output_path, 'heldout_pred_nn_val89.txt'), 'w', encoding='utf-8') as f:
#     [f.write(prediction_string+'\n') for prediction_string in prediction_itos]

# Temp

In [638]:
with open(os.path.join(paths.output_path, 'heldout_pred_nn_'+'fasttext_'+str(3)+'.txt'), 'r', encoding='utf-8') as f:
    test1 = f.read().strip().split('\n')
with open(os.path.join(paths.output_path, 'heldout_pred_nn_'+'residual_lstm_'+str(13)+'_1.txt'), 'r', encoding='utf-8') as f:
    test2 = f.read().strip().split('\n')
with open(os.path.join('..','dev_label.txt'), 'r', encoding='utf-8') as f:
    test3 = f.read().strip().split('\n')

In [639]:
sum([a==b for a, b in zip(test1, test3)])/2000

0.807

In [303]:
def collate_lines_for_test(seq_list, lens):
        inputs = seq_list.permute(1,0).cpu().numpy()
    #     lens = [len(seq) for seq in inputs]
        # sort by length
        seq_order = sorted(range(len(lens)), key=lens.__getitem__, reverse=True)
        ordered_inputs = torch.tensor([inputs[i] for i in seq_order]).permute(1,0).cuda()
        ordered_seq_lens = torch.tensor([lens[i] for i in seq_order]).cuda()
        reverse_order = sorted(range(len(lens)), key=seq_order.__getitem__, reverse=False)
        return ordered_inputs, ordered_seq_lens, reverse_order

In [454]:
original_inputs, original_lengths = next(iter(test_iterator)).text

In [455]:
original_inputs.permute(1,0)

tensor([[   11,   105,    12,    16,     8,     7,    80,    21,    13,  6550,
           589,     3,  1641,  6608,    20,  5757,   492,  1088,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1],
        [  447

In [456]:
test_inputs, ordered_seq_lengths, reverse_order = collate_lines_for_test(original_inputs, original_lengths)

In [457]:
torch.eq(test_inputs.permute(1,0)[reverse_order].permute(1,0),original_inputs).all()

tensor(1, device='cuda:0', dtype=torch.uint8)

In [461]:
test = ordered_seq_lengths[reverse_order]

In [462]:
test

tensor([ 18,  47, 124,  50], device='cuda:0')

In [459]:
original_lengths

tensor([ 18,  47, 124,  50], device='cuda:0')

In [460]:
ordered_seq_lengths

tensor([124,  50,  47,  18], device='cuda:0')